#Creating a Delta Table in Gold Zone with below Details:
#1.Fetching the highest number of rides by month per driver and  highest number of trips and highest spent customer by month & by the year.
#2.Fetching the top rated driver for by the year. 
#3.Fetching the highest spent customer & highest distance travelled customer.

In [0]:
from pyspark.sql.functions import broadcast
import pyspark.sql.functions as F

In [0]:
df1=spark.read.load("/mnt/Deltalake/silver_Zone/Trip_Transactions_Fact")

In [0]:
df2=spark.read.load("/mnt/Deltalake/silver_Zone/Customer_Dimension")

In [0]:
df3=spark.read.load("/mnt/Deltalake/silver_Zone/driver_Dimension")

In [0]:
df4=spark.read.load("/mnt/Deltalake/silver_Zone/Date_Dimension")

In [0]:
df5=df1.join(broadcast(df2),df1.customer_id==df2.Customer_id)

In [0]:
df5.rdd.getNumPartitions()

Out[7]: 3

In [0]:
df5=df5.select("trip_id","Customer_Name","customer_age","customer_gender","Trip_Date","driver_id","total_distance","total_fare","driver_id")

In [0]:
df6=df5.join(broadcast(df3),df3.driver_id==df5.driver_id)

In [0]:
df6=df6.select("trip_id","Customer_Name","customer_age","customer_gender","Trip_Date","total_distance","total_fare","driver_name","driver_age","driver_gender")

In [0]:
df7=df6.join(df4,df4.date==df6.Trip_Date)

In [0]:
df7.createOrReplaceTempView("df7")

Highest Spent & Highest distance travelled by Customer

In [0]:
df_customer_spent_distance=spark.sql("select customer_name, rank_total_distance,rank_total_fare ,total_distance,total_fare , rank_trips_count ,concat(month,year) as month_year,trips_count from (select customer_name, rank() over(partition by month,year order by total_distance desc)as rank_total_distance, rank() over(partition by month,year order by total_fare  desc) as rank_total_fare, total_distance,total_fare,month,year, rank() over(partition by month,year order by trips_count desc ) as rank_trips_count,trips_count from (select customer_name,sum(total_fare) as total_fare,month,year,sum(total_distance) as total_distance,count(trip_id) as trips_count from df7 group by customer_name,month,year)) where rank_total_distance=1 or rank_total_fare=1 or rank_trips_count=1 order by concat(month,year)")

Highest Rating & Highest Trips travelled by Driver

In [0]:
df_driver_trips=spark.sql("select driver_name, concat(month,year) as year_Month,count_trips from (select driver_name, rank() over(partition by month,year order by count_trips desc )as rank_count_trips,month,year,count_trips from (select driver_name,month,year,count(trip_id) as count_trips from df7 group by driver_name,month,year)) where rank_count_trips=1  order by concat(month,year)")

In [0]:
df8=spark.read.load("/mnt/Deltalake/Bronze/Rewards_Points")

In [0]:
df9=df8.join(df7,df7.trip_id==df8.trip_id)

In [0]:
df10=df9.groupBy("Customer_Name","Month","year").sum("customer_rating")

In [0]:
df11=df10.withColumnRenamed("sum(customer_rating)","Customer_Rating")

In [0]:
df12=df9.groupBy("driver_name","Month","year").sum("driver_rating")

In [0]:
df13=df12.withColumnRenamed("sum(driver_rating)","driver_Rating")

In [0]:
df14=df13.withColumn("rank_driver_rating",F.expr("rank() over(partition by concat(month,Year) order by driver_rating desc)"))

In [0]:
df15=df14.filter(df14.rank_driver_rating==1)

In [0]:
df15.display()

driver_name,Month,year,driver_Rating,rank_driver_rating
Kapil,01,2023,124,1
Aaditya,02,2023,87,1
Kumar,03,2023,121,1
Nikshit,04,2023,115,1
Sam,05,2023,122,1


In [0]:
df12=df11.groupBy("customer_name","month","year").sum("customer_rating")
df12=df12.withColumnRenamed("sum(customer_rating)","customer_rating")
df13=df12.withColumn("rank_customer_rating",F.expr("rank() over(partition by concat(month,Year) order by customer_rating desc)"))

In [0]:
df15=df13.filter(df13.rank_customer_rating==1)

In [0]:
df15.display()

customer_name,month,year,customer_rating,rank_customer_rating
ABC2,01,2023,124,1
ABC6,02,2023,89,1
ABC32,03,2023,120,1
ABC3,04,2023,116,1
ABC0,05,2023,120,1


In [0]:
df_customer_spent_distance.write.saveAsTable("df_customer_spent_distance")